Imports

In [3]:
from datetime import datetime
import numpy as np
from ortools.sat.python import cp_model

Adding the finance_utils function

In [122]:
import yfinance as yf
import pandas as pd
from fredapi import Fred


def get_adj_close_from_stocks(stocks, start_date, end_date):
    """
        Extract Adjusted Close from mentioned stocks on specific dates
        Adj Close => Closing price adjusted 
                    for splits and dividend distributions
    """
    adj_close_df = pd.DataFrame()
    
    for s in stocks:
        data = yf.download(s, start=start_date, end=end_date, auto_adjust=False)
        adj_close_df[s] = data['Adj Close']
    
    return adj_close_df

def get_risk_free_rate():
    """
        Returns the Risk Free Rate based on Federal
        Risk Free Rate => theoretical rate of return received on zero-risk assets
    """
    fred = Fred(api_key="e9048dc2c26dae67bc75a443cd644ce3")
    ten_year_treasury_rate = fred.get_series_latest_release('GS10') / 100
    risk_free_rate = ten_year_treasury_rate.iloc[-1]
    
    return risk_free_rate

def get_maximum_risk(stocks, start_date, end_date):
    if (stocks != []):   
        adj_close_df = get_adj_close_from_stocks(stocks, start_date, end_date)
        log_returns = np.log(adj_close_df / adj_close_df.shift(1)).dropna()

        cov_matrix = log_returns.cov() * 250.8875

        stock_volatilities = np.sqrt(np.diag(cov_matrix))  
        max_risk = max(stock_volatilities)

        return max_risk

Adding the markovitz utils function

In [123]:
def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

def expected_return(weights, log_returns):
    return np.sum(log_returns.mean() * weights) * 252

def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
	p_return = expected_return(weights, log_returns) - risk_free_rate
	return p_return / standard_deviation(weights, cov_matrix)

def negative_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

Defining all test inputs

In [202]:
stocks = ['AAPL', 'DSY.PA', 'MSFT']
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 1, 1)
bounds =[(0, 1), (0, 1), (0, 1)]
risk_rate = 0.25

In [ ]:
from ortools.sat.python import cp_model
import numpy as np
from datetime import datetime
import math

def cpsat_sharpe(stocks: list,
                 start_date: datetime,
                 end_date: datetime,
                 bounds: list,
                 risk_rate: float,
                 risk_free_rate: float = get_risk_free_rate()):
    """
    Optimize a portfolio using CP-SAT by maximizing a surrogate Sharpe ratio.
    
    The surrogate objective is:
    
         score = risk_rate * (Portfolio Expected Return - Risk-free Rate) 
                 - (1 - risk_rate) * (Portfolio Risk)
    
    where:
         Portfolio Expected Return = sum_i (w_i * expected_return_i)
         Portfolio Risk ≈ sum_i (w_i * std_i)
    
    Each weight w_i is an integer representing a percentage (with sum = 100).
    
    We scale floating-point coefficients to integers (using e.g. 10^4).
    """
    
    # Step 1: Acquire historical adjusted-close data for the stocks
    adj_close_df = get_adj_close_from_stocks(stocks, start_date, end_date)
    
    # Step 2: Calculate daily returns and then compute expected returns and standard deviations per asset.
    returns = adj_close_df.pct_change().dropna()
    expected_returns_series = returns.mean()   # Mean return per asset
    std_series = returns.std()                   # Volatility per asset
    
    # Scaling factors to convert floats to integers
    scale_obj = 10**4   # For expected returns
    scale_risk = 10**4  # For risk (volatility)
    
    # Compute scaled coefficients for expected returns (each asset)
    coeffs = []
    for i in range(len(stocks)):
        coeff = expected_returns_series.iloc[i]
        scaled_coeff = int(round(coeff * scale_obj))
        coeffs.append(scaled_coeff)
        print(f"Coefficient for {stocks[i]} (raw): {coeff}, scaled: {scaled_coeff}")
    
    # Compute scaled risk coefficients from asset volatilities
    risk_coeffs = []
    for i in range(len(stocks)):
        risk_val = std_series.iloc[i]
        scaled_risk = int(round(risk_val * scale_risk))
        risk_coeffs.append(scaled_risk)
        print(f"Risk coefficient for {stocks[i]} (raw): {risk_val}, scaled: {scaled_risk}")
    
    # Scale risk-free rate for the same scale as expected returns.
    # Since weights sum to 100, the risk-free "contribution" will be risk_free_rate * 100.
    risk_free_rate_scaled = int(round(risk_free_rate * scale_obj))
    
    # Convert risk_rate into an integer weight using the same scaling.
    # This gives us a trade-off coefficient: risk_rate_int corresponds to the return part,
    # and (scale_obj - risk_rate_int) corresponds to the risk penalty.
    risk_rate = risk_rate / get_maximum_risk(stocks, start_date, end_date)
    risk_rate_int = int(round(risk_rate * scale_obj))
    non_risk_rate_int = scale_obj - risk_rate_int  # equivalent to (1 - risk_rate)*scale_obj
    
    # Setup CP-SAT model.
    model = cp_model.CpModel()
    
    # Define decision variables: each stock's weight (in integer percentage terms)
    weight_vars = []
    for i, (lower_bound, upper_bound) in enumerate(bounds):
        lower = int(lower_bound * 100)
        upper = int(upper_bound * 100)
        w = model.NewIntVar(lower, upper, f"weight_{i}")
        weight_vars.append(w)
    
    # Constraint: Weights must sum to 100%
    model.Add(sum(weight_vars) == 100)
    
    # Define the portfolio's expected return (scaled) as:
    #     portfolio_return = sum_i (w_i * scaled_expected_return_i)
    portfolio_return = sum(weight_vars[i] * coeffs[i] for i in range(len(stocks)))
    
    # Compute portfolio excess return by subtracting the risk-free rate contribution.
    # Since the weights add to 100, the total risk-free contribution is risk_free_rate_scaled * 100.
    portfolio_excess_return = portfolio_return - (risk_free_rate_scaled * 100)
    
    # Define portfolio risk as the weighted sum of individual risk coefficients.
    portfolio_risk = sum(weight_vars[i] * risk_coeffs[i] for i in range(len(stocks)))
    
    # Our surrogate Sharpe objective (linear combination):
    # Maximize: risk_rate_int * (portfolio_excess_return) - non_risk_rate_int * (portfolio_risk)
    objective = risk_rate_int * portfolio_excess_return - non_risk_rate_int * portfolio_risk
    model.Maximize(objective)
    
    # Solve the model.
    solver = cp_model.CpSolver()
    solver.parameters.log_search_progress = True
    solver.parameters.max_time_in_seconds = 10
    
    status = solver.Solve(model)
    if status not in [cp_model.OPTIMAL]:
        return {"error": "Aucun portefeuille faisable trouvé"}
    
    # Recover weights (convert back from percentage)
    weights = [solver.Value(w) / 100.0 for w in weight_vars]
    
    # Compute the actual portfolio expected return and risk (using original, unscaled values)
    final_expected_return = sum(weights[i] * expected_returns_series.iloc[i] for i in range(len(stocks)))
    final_risk = sum(weights[i] * std_series.iloc[i] for i in range(len(stocks)))
    
    # Calculate the actual Sharpe ratio if risk is nonzero.
    if final_risk > 0:
        sharpe_ratio = (final_expected_return - risk_free_rate) / final_risk
    else:
        sharpe_ratio = float('-inf')
    
    best_portfolio = {
        "weights": weights,
        "expected_return": final_expected_return,
        "risk": final_risk,
        "sharpe_ratio": sharpe_ratio
    }
    
    return best_portfolio


In [210]:


# Run the optimization
result = cpsat_sharpe(stocks, start_date, end_date, bounds, risk_rate)
print("\nHello here are the results", result)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Coefficient for AAPL (raw): 0.0018349277863999934, scaled: 18
Coefficient for DSY.PA (raw): 0.001195611679813251, scaled: 12
Coefficient for MSFT (raw): 0.0019716604483516445, scaled: 20
Risk coefficient for AAPL (raw): 0.012570054421735874, scaled: 13
Risk coefficient for DSY.PA (raw): 0.016304536583622044, scaled: 16
Risk coefficient for MSFT (raw): 0.01582434182858358, scaled: 16

Starting CP-SAT solver v9.12.4544
Parameters: max_time_in_seconds: 10 log_search_progress: true
Setting number of workers to 16

Initial optimization model '': (model_fingerprint: 0x7bbe3c3427b0b764)
#Variables: 3 (#ints: 3 in objective)
  - 3 in [0,100]
#kLinear3: 1

Starting presolve at 0.00s
  9.96e-06s  0.00e+00d  [DetectDominanceRelations] 
  9.08e-05s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.18e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  2.96e-06s  0.00e+00d  [DetectDuplicateColumns] 
  4.28e-06s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for sy


/tmp/ipykernel_10895/2576055571.py:33: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = adj_close_df.pct_change().dropna()
